In [2]:
def encontrar_combinacoes(comprimentos, limite):
    solucoes = set()

    def backtrack(combinacao, total):
        if total > limite:
            return

        pode_adicionar = False
        for comp in comprimentos:
            if total + comp <= limite:
                pode_adicionar = True
                break
                
        if not pode_adicionar and total > 0:
            desperdicio = limite - total
            #if tuple(tuple(combinacao + [total] + [desperdicio])) not in solucoes:
             #   print(combinacao)
            solucoes.add(tuple(combinacao + [total] + [desperdicio]))
            return

        for i, comp in enumerate(comprimentos):
            nova_combinacao = combinacao[:]
            nova_combinacao[i] += 1
            backtrack(nova_combinacao, total + comp)

    combinacao_inicial = [0] * len(comprimentos)
    backtrack(combinacao_inicial, 0)

    return solucoes

def printar_solucoes(solucoes):
    for s in solucoes: 
        total = s[-2]
        desperdicio = s[-1]
        partes = []
        for qtd, tamanho in zip(s[:-1], comprimentos):
            partes.append(f"{qtd} barras de {tamanho} metros")
        frase = " + ".join(partes)
        print()
        print(f"{frase} | total = {total} metros | desperdicio = {desperdicio} metros")

def montar_desperdicio(solucoes):
    desperdicio = []
    for s in solucoes:
        desperdicio.append(s[-1])
    return desperdicio

def montar_matriz_respostas(solucoes):
    return transpor_matriz([list(s[:-2]) for s in solucoes])

def transpor_matriz(matriz):
    linhas = len(matriz)
    colunas = len(matriz[0])
    transposta = [[0 for _ in range(linhas)] for _ in range(colunas)]
    for i in range(linhas):
        for j in range(colunas):
            transposta[j][i] = matriz[i][j]
    return transposta

In [3]:
comprimentos = [50, 60, 80]
limite = 150

solucoes = encontrar_combinacoes(comprimentos, limite)

desperdicio = montar_desperdicio(solucoes)
n = len(desperdicio)
print(desperdicio)

limitantes = [120, 100, 70]

coeficientes = montar_matriz_respostas(solucoes)

[30, 40, 20, 10, 0]


In [4]:
%pip install ortools

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from ortools.linear_solver import pywraplp

load c:\Users\ptimb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\ptimb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\ptimb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\ptimb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\ptimb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\ptimb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\ptimb\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\ortools.dll...


In [6]:
solver = pywraplp.Solver.CreateSolver('SCIP') #declara o solver: SCIP para PLI e GLOP para PL
infinity = solver.infinity()

In [7]:
x = []
for i in range(len(coeficientes[0])):
    x.append(solver.IntVar(0.0, infinity, 'p' + str(i+1)))

In [8]:
objetivo = solver.Objective()
for i in range(n):
  objetivo.SetCoefficient(x[i], desperdicio[i])
objetivo.SetMinimization()

In [9]:
for i in range(len(coeficientes)):
  restricao = solver.Constraint(limitantes[i],infinity)
  for j in range(len(coeficientes[i])):
    restricao.SetCoefficient(x[j], coeficientes[i][j])

In [12]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
  print('Solução:')
  print('Valor da função objetivo =', solver.Objective().Value())
  for i in range(n):
    print(x[i].name(), '=', x[i].solution_value())
printar_solucoes(solucoes)

Solução:
Valor da função objetivo = 1000.0
p1 = 0.0
p2 = 0.0
p3 = 0.0
p4 = 100.0
p5 = 40.0

0 barras de 50 metros + 2 barras de 60 metros + 0 barras de 80 metros | total = 120 metros | desperdicio = 30 metros

1 barras de 50 metros + 1 barras de 60 metros + 0 barras de 80 metros | total = 110 metros | desperdicio = 40 metros

1 barras de 50 metros + 0 barras de 60 metros + 1 barras de 80 metros | total = 130 metros | desperdicio = 20 metros

0 barras de 50 metros + 1 barras de 60 metros + 1 barras de 80 metros | total = 140 metros | desperdicio = 10 metros

3 barras de 50 metros + 0 barras de 60 metros + 0 barras de 80 metros | total = 150 metros | desperdicio = 0 metros


In [13]:
print(solver.ExportModelAsLpFormat(False))

\ Generated by MPModelProtoExporter
\   Name             : 
\   Format           : Free
\   Constraints      : 3
\   Variables        : 5
\     Binary         : 0
\     Integer        : 5
\     Continuous     : 0
Minimize
 Obj: +30 p1 +40 p2 +20 p3 +10 p4 
Subject to
 auto_c_000000000: +1 p2 +1 p3 +3 p5  >= 120
 auto_c_000000001: +2 p1 +1 p2 +1 p4  >= 100
 auto_c_000000002: +1 p3 +1 p4  >= 70
Bounds
 0 <= p1 <= inf
 0 <= p2 <= inf
 0 <= p3 <= inf
 0 <= p4 <= inf
 0 <= p5 <= inf
Generals
 p1
 p2
 p3
 p4
 p5
End

